In [1]:
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
import json
import matplotlib
import os
import random
import torch

from collections import Counter
from itertools import islice
from sklearn.utils import shuffle
from tqdm import tqdm, trange
from torch.optim import Adam
from torch.utils.data import TensorDataset

from models.Encoder import *
from models.Decoder import *
from models.utils import *
from utils import *

# set fixed random seed to reproduce results
np.random.seed(42)
random.seed(42)

%matplotlib inline

GPU is available
GPU is available


Using TensorFlow backend.


# Experiment 1b

In [2]:
# define experiment
exp='/exp_1'
# define number of iterations
n_iters = 20000
# define batch size
batch_size = 32

In [3]:
device = ("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
# load dataset into memory, and get w2idx, idx2w, w2freq dictionaries and lists of input and output sentences
cmd_vocab, w2i_cmds, i2w_cmds, cmds_train, act_vocab, w2i_acts, i2w_acts, acts_train = load_dataset(exp=exp, split='/train')
_, _, _, cmds_test, _, _, _, acts_test = load_dataset(exp=exp, split='/test')

In [5]:
## create input and output language pairs ##

# training
train_cmd_act_pairs = create_pairs(cmds_train, acts_train)
print("Number of train source-target pairs: {}".format(len(train_cmd_act_pairs)))

# testing
test_cmd_act_pairs = create_pairs(cmds_test, acts_test)
print("Number of test source-target pairs: {}".format(len(test_cmd_act_pairs)))

Number of train source-target pairs: 16728
Number of test source-target pairs: 4182


In [6]:
# show random train command-action pair
random_pair = random.choice(train_cmd_act_pairs)
print("Command: {}".format(random_pair[0]))
print("Action: {}".format(random_pair[1]))

Command: ['look', 'left', 'thrice', 'after', 'turn', 'opposite', 'left']
Action: ['I_TURN_LEFT', 'I_TURN_LEFT', 'I_TURN_LEFT', 'I_LOOK', 'I_TURN_LEFT', 'I_LOOK', 'I_TURN_LEFT', 'I_LOOK']


In [7]:
# show random test command-action pair
random_pair = random.choice(test_cmd_act_pairs)
print("Command: {}".format(random_pair[0]))
print("Action: {}".format(random_pair[1]))

Command: ['run', 'right', 'thrice', 'after', 'jump', 'right']
Action: ['I_TURN_RIGHT', 'I_JUMP', 'I_TURN_RIGHT', 'I_RUN', 'I_TURN_RIGHT', 'I_RUN', 'I_TURN_RIGHT', 'I_RUN']


In [8]:
cmds_train, acts_train, input_lengths_train, masks = pairs2idx(cmds_train, acts_train, w2i_cmds, w2i_acts, padding=True, training=True)
cmds_test, acts_test, input_lengths_test = pairs2idx(cmds_test, acts_test, w2i_cmds, w2i_acts, padding=True, training=False)

In [9]:
# create train and test data loaders
train_dl = create_batches(cmds_train, acts_train, input_lengths_train, batch_size=batch_size, masks=masks, split='train', num_samples=n_iters)
test_dl = create_batches(cmds_test, acts_test, input_lengths_test, batch_size=batch_size, split='test')

In [10]:
### Hyperparameters for training ###

# source language (i.e., commands) vocabulary size |V_source|
in_size = len(w2i_cmds)

# target language (i.e., actions) vocabulary size |V_target|
out_size = len(w2i_acts)

# size of word embeddings
emb_size = 20

# size of hidden units
hidden_size = 100

# number of layers
n_layers = 2

# learning rate
lr = 1e-3

# layer dropout rate
dropout_p = 0.3

# define whether encoder is uni- or bidirectional
bidir = False

# number of epochs
n_epochs = 8 # 7-10 epochs (20.000 iterations each) seem to be sufficient to learn the mapping

In [11]:
## Instantiate models ##

encoder = EncoderLSTM(in_size, emb_size, hidden_size, n_layers, dropout=dropout_p, bidir=bidir)
decoder = DecoderLSTM(emb_size, hidden_size, out_size, n_layers, dropout=dropout_p)

In [12]:
#encoder = EncoderGRU(in_size, emb_size, hidden_size, n_layers, dropout=dropout_p, bidir=True)
#decoder = DecoderGRU(emb_size, hidden_size, out_size, n_layers, dropout=dropout_p)

In [13]:
encoder.cuda()

EncoderLSTM(
  (embedding): Embedding(16, 20, padding_idx=0)
  (lstm): LSTM(20, 100, num_layers=2, batch_first=True, dropout=0.3, bidirectional=True)
)

In [14]:
# move models to GPU, if GPU is available (for faster computation)
decoder.cuda()

DecoderLSTM(
  (embedding): Embedding(9, 20, padding_idx=0)
  (lstm): LSTM(20, 100, num_layers=2, batch_first=True, dropout=0.3)
  (linear): Linear(in_features=100, out_features=9, bias=True)
)

## Training (1a)

In [15]:
train_losses, train_accs, encoder, decoder = train(train_dl,
                                                   w2i_cmds, w2i_acts,
                                                   i2w_cmds, i2w_acts,
                                                   encoder, decoder,
                                                   epochs=n_epochs,
                                                   batch_size=batch_size,
                                                   learning_rate=lr)

Epoch:   0%|                                                                                     | 0/8 [00:00<?, ?it/s]

Loss: nan
Acc: 0.00031146179401993354

Command: run opposite right twice after walk opposite right twice <EOS>
True action: I_TURN_RIGHT I_TURN_RIGHT I_WALK I_TURN_RIGHT I_TURN_RIGHT I_WALK I_TURN_RIGHT I_TURN_RIGHT I_RUN I_TURN_RIGHT I_TURN_RIGHT I_RUN <EOS>
Pred action: I_TURN_RIGHT I_TURN_RIGHT I_TURN_RIGHT I_TURN_RIGHT I_WALK I_TURN_RIGHT I_TURN_RIGHT I_WALK I_TURN_RIGHT I_TURN_RIGHT I_RUN I_RUN <EOS>

True sent length: 13
Pred sent length: 13



KeyboardInterrupt: 

## Testing (1a)

In [ ]:
test_acc = test(test_dl,
                w2i_cmds, w2i_acts, 
                i2w_cmds, i2w_acts, 
                encoder, decoder,
                batch_size=batch_size)

## Train and test over different numbers of distinct source-target pairs (1b)

In [ ]:
## experiment 1b train and test loop over pre-defined ratios ##

ratios = np.array([0.01, 0.02, 0.04, 0.08, 0.16, 0.32, 0.64])
accs_per_ratio = {}

for ratio in ratios:
    train_samples = sample_distinct_pairs(train_cmd_act_pairs, ratio)
    print("-------------------------------------------------")
    print("Current percentage of total commands used for training: {}%".format(ratio*100))
    print("Number of distinct examples shown during training: {}".format(len(train_samples)))
    print("-------------------------------------------------")
    # instantiate new encoder and decoder for each ratio
    encoder = EncoderLSTM(in_size, emb_size, hidden_size, n_layers)
    decoder = DecoderLSTM(emb_size, hidden_size, out_size, n_layers)
    encoder.cuda()
    decoder.cuda()
    train_losses, train_accs, encoder, decoder = train(train_samples, w2i_cmds, w2i_acts, i2w_cmds, i2w_acts, encoder, decoder, n_epochs)
    test_acc = test(test_cmd_act_pairs, w2i_cmds, w2i_acts, i2w_cmds, i2w_acts, encoder, decoder)
    accs_per_ratio[ratio] = test_acc

In [ ]:
# save results in .json file
with open('./results/experiment_1b.json', 'w') as json_file:
      json.dump(accs_per_ratio, json_file)

In [ ]:
# load results from experiment 1b
with open('./results/experiment_1b.json') as json_file:
    accs_per_ratio = json.load(json_file)

In [ ]:
ratios, test_accs = zip(*accs_per_ratio.items())
plt.bar(ratios, test_accs)
plt.xlabel('Percent of distinct commands used for training')
plt.ylabel('Test accuracy on unseen commands (%)')
plt.xticks(ticks=ratios, labels=list(map(lambda rat: str(int(float(rat) * 100)) + '%', ratios)))
plt.title('Experiment 1b')
plt.savefig('./Paper/Plots/experiment_1b.png')
plt.show()
plt.clf()